In [1]:
# -*- coding: utf-8 -*-
import sys
from pymarc import Field, Reader, Record, MARCWriter
!pip install elasticsearch==7.6.0
!pip install pymongo
!pip install pymarc

You should consider upgrading via the '/Users/vyacheslavtykhonov/.pyenv/versions/3.7.0/envs/jupyter3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/vyacheslavtykhonov/.pyenv/versions/3.7.0/envs/jupyter3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/vyacheslavtykhonov/.pyenv/versions/3.7.0/envs/jupyter3/bin/python -m pip install --upgrade pip' command.


In [2]:
!curl 'https://storage.googleapis.com/kaggle-data-sets/551982%2F1359228%2Fcompressed%2Fmetadata.csv.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1595663670&Signature=nviUFqLPLOXX32rNnjQoali3UKgteYSsvdqzJPFfJ8WSILtHMSqU7C5seb%2FeAc96fJkmdBOf5OoVWwpHwt4r%2FnOUt3NZkxf63aR6PLh8vieD4CloYtqGOEIb8Pg6wNG3LwUD8fIE0jR42WjYa%2FVkYshsBASR1Mz8D6z2TW1DPIDp%2Bdjw%2BEXKjRqjQ%2FXdN1hDjZ2uoQ%2FmCWdZAf83G%2B716TujB5G4LrlA23ReGVxlFZLLeycnRTeYrkY3TrDZVSTwpAWKiC5pRe7aeQ4i87I3%2BpQj0RO%2FxRWI2ZzSlacbfDr0lq8EpLQ52jSmkEcd%2F2zN4qWKRClNbhmScJp8Mkvf%2BA%3D%3D' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Referer: https://www.kaggle.com/' -H 'Connection: keep-alive' -H 'Upgrade-Insecure-Requests: 1' -H 'TE: Trailers' -o compressed_metedata.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 94.7M  100 94.7M    0     0  14.3M      0  0:00:06  0:00:06 --:--:-- 15.2M


In [3]:
!rm metadata.csv
!unzip compressed_metedata.zip

rm: metadata.csv: No such file or directory
Archive:  compressed_metedata.zip
  inflating: metadata.csv            


In [4]:
import pandas as pd
metadata = pd.read_csv('metadata.csv')
metadata.info()

/Users/vyacheslavtykhonov/.pyenv/versions/jupyter3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195711 entries, 0 to 195710
Data columns (total 19 columns):
cord_uid            195711 non-null object
sha                 82517 non-null object
source_x            195711 non-null object
title               195655 non-null object
doi                 139395 non-null object
pmcid               87299 non-null object
pubmed_id           119076 non-null object
license             195711 non-null object
abstract            139874 non-null object
publish_time        195693 non-null object
authors             189141 non-null object
journal             184692 non-null object
mag_id              0 non-null float64
who_covidence_id    51738 non-null object
arxiv_id            2500 non-null object
pdf_json_files      82517 non-null object
pmc_json_files      63394 non-null object
url                 153320 non-null object
s2_id               167757 non-null float64
dtypes: float64(2), object(17)
memory usage: 28.4+ MB


In [5]:
metadata.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,1.14726e+07,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,1.1668e+07,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,1.1668e+07,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,1.16869e+07,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,1.16869e+07,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [6]:
item = metadata.ix[0]
len(metadata.index)

/Users/vyacheslavtykhonov/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


195711

In [63]:
def cordmarc(metadata):
	title = metadata['title']
	authors = ''
	try:
		authors = metadata['authors'].split('; ')
	except:
		authors = ''
	abstract = metadata['abstract']    
	isbn = metadata['cord_uid']
	imprint = metadata['pmcid']
	imprint=''
	journal = metadata['journal']
	license = metadata['license']
	year = str(metadata['publish_time'])[0:4]
	ctrl = metadata['cord_uid'] #findvalue(rownum, 2)
	language = 'eng'    
	doi = "https://doi.org/%s" % metadata['doi']
	subject = 'Medicine'
	tags = ['COVID-19', 'coronavirus']    
	affiliations = ["CoronaWhy Labs"]
    
	record = Record()#to_unicode=True)
	
	#store vendor catalog id in 001
	field = Field(tag='001', data=ctrl)
	record.add_field(field)
	
	#008
	oh8 = list('/' * 39)
	oh8[6:8] = 'cas'
	oh8[35:37] = 'chi'
	oh8[15:17] = 'cc '
	field = Field(tag='008', data="".join(oh8))
	record.add_field(field)
	
	#title
	field = Field('245', ['1', '0'], ['a', title])
	record.add_field(field)
	#leader
	l = list(record.leader)
	l[4] = 'i'
	l[5] = 'c'
	l[6] = 'a'
	l[7] = 's'
	l[9] = 'a' # flag saying this record is utf8
	record.leader = "".join(l)
	
	if doi: 
		field = Field('856', ['40', ''], ['u', doi])
		record.add_field(field)
	#tags
	for tag in tags:
		field = Field('651', [' ', ' '], ['x', tag])
		record.add_field(field)
	
	field = Field(
		tag = '260',
		indicators = ['\\', '\\'],
		subfields = [
			'b', imprint, 
			'c', year
			]
		)
	record.add_field(field)
	if isbn:
		field = Field('020', [' ', ' '], ['a', isbn])
		record.add_field(field)
	
	#language
	field = Field('041', [' ', ' '], ['a', language])    
	record.add_field(field)
	
	#author
	for a in authors:
		field = Field(
			tag = '100',
		indicators = ['0', ' '],
		subfields = [
			'a', a, 
			'u', ' '
			]
		)
		record.add_field(field)
	
	#subject
	field = Field('650', ['00', ' '], ['a', subject])    
	record.add_field(field)
	
	if journal:
		field = Field('260', ['1', '0'], ['b', journal])
		record.add_field(field)        
	#title
	if title:
		field = Field('245', ['1', '0'], ['a', title])
		record.add_field(field)
	# 538			|a Mode of access: World Wide Web. 
	# extra fields
	field = Field('538', [' ', ' '], ['a', 'Mode of access: World Wide Web'])
	record.add_field(field)
	    
	#affiliation 852$a
	for aff in affiliations:
		field = Field(
			tag = '710',
		indicators = ['2', ' '],
		subfields = [
			'a', aff, 
			'b', 'Labs'
			]
		)
		record.add_field(field)
	# license
	if license:    
		field = Field('900', [' ', ' '], ['a', license])
		record.add_field(field)
	if abstract:
		field = Field('520', ['1', '0'], ['a', abstract])
		record.add_field(field)
	return record

In [53]:
OUTFILE = '/tmp/marc/cord01.mrc'
writer = MARCWriter(open(OUTFILE,'wb'))
#for i in reversed(range(len(metadata.index))):
for i in metadata.index[0:1]:
	record=cordmarc(metadata.ix[1])
	print(record.as_marc())
	#write the record to file
	#writer.write(record)
	try:
		xcheck = record.as_marc()
		writer.write(record)
	except:
		#print(record.as_marc())
		continue
writer.close()
print('Ready!')

b'01732cas a2200253   4500001000900000008004300009245006300052856003300115651001300148651001600161260001100177020001300188041000800201100003200209100002800241100002700269650001400296260001500310245006300325538003500388710002500423900001000448520102000458\x1e02tnwd4m\x1e//////cas//////cc //////////////////chi///\x1e10\x1faNitric oxide: a pro-inflammatory mediator in lung disease?\x1e40\x1fuhttps://doi.org/10.1186/rr14\x1e  \x1fxCOVID-19\x1e  \x1fxcoronavirus\x1e\\\\\x1fb\x1fc2000\x1e  \x1fa02tnwd4m\x1e  \x1faeng\x1e0 \x1faVliet, Albert van der\x1fuLabs\x1e0 \x1faEiserich, Jason P\x1fuLabs\x1e0 \x1faCross, Carroll E\x1fuLabs\x1e00 \x1faMedicine\x1e10\x1fbRespir Res\x1e10\x1faNitric oxide: a pro-inflammatory mediator in lung disease?\x1e  \x1faMode of access: World Wide Web\x1e2 \x1faCoronaWhy Labs\x1fbLabs\x1e  \x1fano-cc\x1e10\x1faInflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO\xe2\x80\xa2) and increased indices of NO\x

/Users/vyacheslavtykhonov/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """


In [ ]:
l = len(metadata.index)
count = 1000
pages = int(l / count) + 1
commands = []
for page in range(0,l):
	OUTFILE = "/tmp/marc/cord%s.mrc" % page
	fromres = page * count
	tores = (page + 1) * count
	print("%s %s %s" % (fromres, tores, OUTFILE))
	writer = MARCWriter(open(OUTFILE,'wb'))
	cmd = "./import-marc.sh /tmp/marc/cord%s.mrc" % page
	commands.append(cmd)
	for i in metadata.index[fromres:tores]:
		record=cordmarc(metadata.ix[i])
#		print(record.as_marc())
		try:
			xcheck = record.as_marc()
			writer.write(record)
		except:
			#print(record.as_marc())
			continue
	writer.close()

0 1000 /tmp/marc/cord0.mrc


/Users/vyacheslavtykhonov/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


1000 2000 /tmp/marc/cord1.mrc
2000 3000 /tmp/marc/cord2.mrc
3000 4000 /tmp/marc/cord3.mrc
4000 5000 /tmp/marc/cord4.mrc
5000 6000 /tmp/marc/cord5.mrc
6000 7000 /tmp/marc/cord6.mrc
7000 8000 /tmp/marc/cord7.mrc
8000 9000 /tmp/marc/cord8.mrc
9000 10000 /tmp/marc/cord9.mrc
10000 11000 /tmp/marc/cord10.mrc
11000 12000 /tmp/marc/cord11.mrc
12000 13000 /tmp/marc/cord12.mrc
13000 14000 /tmp/marc/cord13.mrc
14000 15000 /tmp/marc/cord14.mrc
15000 16000 /tmp/marc/cord15.mrc
16000 17000 /tmp/marc/cord16.mrc
17000 18000 /tmp/marc/cord17.mrc
18000 19000 /tmp/marc/cord18.mrc
19000 20000 /tmp/marc/cord19.mrc
20000 21000 /tmp/marc/cord20.mrc
21000 22000 /tmp/marc/cord21.mrc
22000 23000 /tmp/marc/cord22.mrc
23000 24000 /tmp/marc/cord23.mrc
24000 25000 /tmp/marc/cord24.mrc
25000 26000 /tmp/marc/cord25.mrc
26000 27000 /tmp/marc/cord26.mrc
27000 28000 /tmp/marc/cord27.mrc
28000 29000 /tmp/marc/cord28.mrc
29000 30000 /tmp/marc/cord29.mrc
30000 31000 /tmp/marc/cord30.mrc
31000 32000 /tmp/marc/cord31.mrc
320

In [65]:
f = open("/tmp/marc/runall.sh", "w")
for cmd in commands:
    f.write("%s\n" % cmd)
f.close()